# SQL Queries and Data Visualization #
 
Welcome to this SQL tutorial using Python. Our focus here is on executing SQL queries, and for this purpose, we've set up a local database using SQLite3. SQL stands out for its efficiency in handling data queries. Additionally, we'll demonstrate how to integrate our database with a data visualization tool like PowerBI, enriching our analysis capabilities.

In this tutorial, we use various time series datasets: (1) Job destruction data from the Census' Business Dynamics Statistics; (2) The U.S. GDP, unemployment, and mortgage interest rates from the Federal Reserve Bank of St. Louis; (3) Average personal consumption and income by state from the U.S. Bureau of Economic Analysis; (3) Natality data by state and age group from the Centers for Disease Control and Prevention; (4) Mass layoff notices from the Federal Reserve Bank of Cleveland.

Our goal is to retrieve, process, and utilize this data effectively. Once we format the data as required, we'll showcase how to use it with PowerBI to create an insightful dashboard.

## Connecting to the Database ##

The first step is to establish a connection with our database and explore its structure, including the available tables and their respective columns. We'll begin by importing the necessary libraries and then connect to the SQLite database. Following this, we execute a SQL query to list all the available tables in the database. This query will return a set of rows, where each row contains the name of one table.

In [57]:
import pandas as pd
import numpy as np
import sqlite3
import csv

In [58]:
conn = sqlite3.connect('C:\GitHub\labor_market_natality.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
conn.close()

table_names = [table[0] for table in tables]
print(table_names)

['bds_state', 'gdp_us', 'mortagage_us', 'cons_state', 'inc_state', 'unemployment_us', 'natality_state', 'warn_state_data', 'combined_state_data']


## Checking Columns ##

In this section, we will develop a Python function that retrieves the names and data types of columns for each table in our database. This understanding is essential for determining how tables can be combined and which transformations are necessary to obtain meaningful insights.

## Function to Retrieve Column Information ##

We'll create a function named get_column_info. This function will take the name of a table as an input and return a list of tuples, each containing the name and data type of a column in that table. This approach provides a clear view of the table's structure, aiding in our data analysis and preparation process.

In [64]:
def check_columns(table):
    conn = sqlite3.connect('C:\GitHub\labor_market_natality.db')
    cursor = conn.cursor()
    cursor.execute('PRAGMA table_info('+table+')')
    columns = cursor.fetchall()
    conn.close()
    column_names = [column[1] for column in columns]
    column_types = [column[2] for column in columns]
    print(column_names)
    print(column_types)

In [65]:
check_columns('bds_state')

['index', 'statefips', 'year', 'ajd_total', 'jdd_total', 'ajd_female', 'ajd_male', 'jdd_female', 'jdd_male', 'pop_total', 'pop_female', 'pop_male']
['INTEGER', 'REAL', 'INTEGER', 'INTEGER', 'INTEGER', 'REAL', 'REAL', 'REAL', 'REAL', 'INTEGER', 'INTEGER', 'INTEGER']


In [35]:
check_columns('natality_state')

['index', 'statefips', 'year', 'month', 'age', 'births', 'quarter']
['INTEGER', 'INTEGER', 'INTEGER', 'INTEGER', 'TEXT', 'TEXT', 'TEXT']


In [36]:
check_columns('warn_state_data')

['index', 'month', 'year', 'quarter', 'statefips', 'layoffs']
['INTEGER', 'INTEGER', 'TEXT', 'TEXT', 'TEXT', 'REAL']


In [37]:
check_columns('cons_state')

['statefips', 'year', 'durable_exp', 'all_exp']
['REAL', 'TEXT', 'REAL', 'REAL']


In [38]:
check_columns('inc_state')

['statefips', 'year', 'quarter', 'personal', 'population']
['REAL', 'REAL', 'REAL', 'INTEGER', 'INTEGER']


## Combining and Transforming Data ##

With the column information at our disposal, we can strategically decide how to combine data from different tables. This step is crucial for creating a comprehensive dataset that can be effectively used in data visualization tools like PowerBI.

We will use SQL commands to create a new table in our database. This table will be specifically structured to hold the combined and transformed data, optimizing it for visualization purposes. We'll ensure that the table schema (i.e., frequency, column names and data types) aligns with the requirements of our analysis and the capabilities of PowerBI.

## Connecting PowerBI to the SQLite Database ##

With our new table ready, the next crucial step is to establish a connection between PowerBI and our SQLite database. This direct connection enables us to use the live data from the database, facilitating real-time analysis and interactive dashboard creation.

## Creating Yearly Time Series ##

Having gained detailed insights into the structure of our state-level data tables, we've identified a challenge: the data frequencies vary across different tables. For instance, WARN notices and natality data are available monthly, whereas personal income data is quarterly, and both job destruction and per-capita consumption data are presented annually. To effectively analyze and visualize these datasets together, we need to standardize their frequencies to a yearly basis.

## Standardizing Data Frequencies ##

(1) Monthly Data (WARN Notices and Natality): We will aggregate these monthly datasets by summing the data for each year. This process will give us the total annual figures for WARN notices and natality, allowing for a yearly comparison; (2) Quarterly Data (Personal Income): Since this dataset is already presented as a rate, we will calculate the annual average. This method ensures that we capture the typical income trend for each year, smoothing out any seasonal variations; (3) Yearly Data (Job Destruction and Per-Capita Consumption): These datasets are already in the desired yearly format and will be used as is in our analysis.

## Combining Datasets for a Unified View ##

After processing the individual datasets, our next step is to combine them into a single, comprehensive yearly dataset. This unified dataset will include yearly figures for natality, WARN notices, personal income, job destruction, and per-capita consumption.

We start by processing and combining the WARN notices and natality data:

In [39]:
conn = sqlite3.connect(r'C:\GitHub\labor_market_natality.db')
cursor = conn.cursor()

In [40]:
natality_data = """
    (SELECT statefips, year, SUM(births) as total_births 
    FROM natality_state 
    GROUP BY statefips, year) n
"""

In [41]:
warn_data = """
    (SELECT statefips, year, SUM(layoffs) as total_layoffs 
    FROM warn_state_data 
    GROUP BY statefips, year) w
"""

In [42]:
create_table_query = f"""
    CREATE TABLE combined_state_data AS 
    SELECT 
        n.statefips, 
        n.year, 
        n.total_births, 
        w.total_layoffs 
    FROM {natality_data} 
    LEFT JOIN {warn_data} 
    ON n.statefips = w.statefips AND n.year = w.year
"""

In [43]:
cursor.execute(create_table_query)

Now, we turn our attention to the income data, applying a similar process of transformation and integration. The goal here is to refine the income data into a state-by-year format, aligning it with the structure of the dataset we've already started building.

In [44]:
cursor.execute("ALTER TABLE combined_state_data ADD COLUMN total_pop INTEGER;")
cursor.execute("ALTER TABLE combined_state_data ADD COLUMN avg_personal_inc REAL;")

In [45]:
inc_data = """
    SELECT statefips, year, AVG(population) as total_pop, AVG(personal) as avg_personal_inc 
    FROM inc_state 
    GROUP BY statefips, year
"""

In [46]:
bring_inc_data = f"""
    UPDATE combined_state_data
    SET 
        total_pop = (SELECT w.total_pop 
            FROM ({inc_data}) w 
            WHERE combined_state_data.statefips = w.statefips AND combined_state_data.year = w.year),
            
        avg_personal_inc = (SELECT w.avg_personal_inc 
            FROM ({inc_data}) w 
            WHERE combined_state_data.statefips = w.statefips AND combined_state_data.year = w.year);
"""

In [47]:
cursor.execute(bring_inc_data)

In [48]:
cursor.execute("ALTER TABLE combined_state_data ADD COLUMN all_exp REAL;")
cursor.execute("ALTER TABLE combined_state_data ADD COLUMN durable_exp REAL;")
cursor.execute("ALTER TABLE combined_state_data ADD COLUMN ajd_total REAL;")

In [49]:
bring_yearly_series = """
    UPDATE combined_state_data
    SET 
        durable_exp = (SELECT w.durable_exp 
            FROM cons_state w 
            WHERE combined_state_data.statefips = w.statefips AND combined_state_data.year = w.year),
        all_exp = (SELECT w.all_exp 
            FROM cons_state w 
            WHERE combined_state_data.statefips = w.statefips AND combined_state_data.year = w.year),
        ajd_total = (SELECT w.ajd_total 
            FROM bds_state w 
            WHERE combined_state_data.statefips = w.statefips AND combined_state_data.year = w.year);
"""

In [50]:
cursor.execute(bring_yearly_series)

With all the state-level data now combined into a single table, our next step is to conduct a thorough verification of this new dataset. This process is critical to ensure that our data integration efforts have resulted in an accurate and reliable dataset. We will transform our newly created SQL table into a Pandas DataFrame. This transformation makes it easier to visually inspect the data and perform any necessary checks. By examining the DataFrame, we can quickly assess whether the data has been combined correctly. We'll look for inconsistencies, alignment of data across different years and states, and any anomalies that might indicate issues in the data processing or merging steps.

In [51]:
query = "SELECT * FROM combined_state_data;"
df = pd.read_sql_query(query, conn)
df

,statefips,year,total_births,total_layoffs,total_pop,avg_personal_inc,all_exp,durable_exp,ajd_total
0,1,2003,59518.0,12704.0,NaN,NaN,102106.6,13427.5,-3886.0
1,1,2004,59460.0,8961.0,NaN,NaN,108428.7,14203.4,-4794.0
2,1,2005,60399.0,5530.0,NaN,NaN,115631.1,14885.3,-7282.0
3,1,2006,63165.0,9734.0,NaN,NaN,121487.4,15346.7,-9176.0
4,1,2007,64733.0,10342.0,NaN,NaN,126916.4,15862.2,-37928.0
...,...,...,...,...,...,...,...,...,...
1015,56,2018,6523.0,NaN,575109.50,59163.25,24652.3,2602.9,2467.0
1016,56,2019,6517.0,NaN,575605.50,61834.75,25235.7,2621.4,397.0
1017,56,2020,6084.0,NaN,577557.00,65562.75,25579.8,2783.0,-655.0
1018,56,2021,6229.0,NaN,579429.75,70529.25,28312.5,3278.2,31.0


In [52]:
conn.commit()
conn.close()

## Moving to PowerBI ##

With our comprehensive table of different time series data ready, the next exciting step is to utilize PowerBI for creating a sample dashboard. PowerBI's robust visualization capabilities will allow us to explore and present our data in dynamic and insightful ways.

To access SQLite3 tables in PowerBI, the first step is to install an ODBC driver. This driver acts as a bridge between SQLite and PowerBI, enabling them to communicate effectively. You can download a suitable driver (for example, http://www.ch-werner.de/sqliteodbc). Once the driver is installed, start PowerBI and click on "Get Data".

    Selecting the Data Source: C
    Setting Up the Connection: 

    Authentication: 

    Selecting the Table: 

![step1](Step1.png)


Then choose "More" to view all data sources and then find "ODBC" in the list. Upon selecting "ODBC", choose "SQLite3 Datasource" and proceed to "Advanced options".


![step2](Step2A.png)


In the "Connection string" field, enter the path to the SQLite3 database you created. For example, if your database is located at C:\GitHub\labor_market_natality.db, your connection string would be database=C:\GitHub\labor_market_natality.db.


![step3](Step3.png)

![step4](Step4.png)


When prompted for credentials, select 'current credentials'. PowerBI will then display a list of tables from your database. For this tutorial, select the 'combined_state_data' table, which is the one we prepared earlier.

Now, let's dive into the process of creating a data visualization in PowerBI. We will start by creating a line chart. To do that, select the "Line Chart" option. This type of chart is particularly effective for displaying trends over time. For a time series plot, the X-axis typically represents the time variable. In our case, drag the "year" variable from your dataset to the X-axis field in the line chart settings. This will set up the foundation of our plot, showing the progression of time on the horizontal axis. The Y-axis will represent the data we want to analyze over time. For example, if you want to visualize trends in durable goods, drag the "durable goods" variable to the Y-axis field. This will plot the values of durable goods against the corresponding years, allowing us to observe changes and trends in this variable over time.

![step6](Step6.png)

To further refine your visualization in PowerBI and make it more user-friendly, you can customize labels, formats, and filters. Right-click on the variable in the Y-axis field and select "Rename for this visual". For more detailed customization, like changing titles, adjusting colors, or modifying other general parameters, navigate to "Format your visual". This section in PowerBI offers a range of options to tailor your chart’s appearance, including font sizes, line styles, and color palettes. To focus on a specific time period, such as 2010-2020, or to filter the data in any way, use the filter options. In the "Filters on this visual" section, add a filter, for example, for the state variable. This allows you to isolate and explore data relevant to a particular state, offering more targeted insights.

![step7](Step7.png)

After following the outlined steps, I successfully created a basic dashboard in PowerBI that showcases some aspects of our dataset. Since our data is organized by state and year, I specifically focused on creating these visualizations for California, Texas, and New York. This state-level analysis allows for a more detailed examination of trends and patterns within these specific regions. The first plot illustrates the relationship between the number of births and spending on durable goods over time. The second time series plot compares the number of births with the total population. I also created a scatterplot that explores the relationship between spending on durable goods and the incidence of mass layoffs. This visualization can reveal interesting patterns or correlations between economic conditions and employment dynamics.

![step8](Step8.png)

This tutorial provides a concise yet comprehensive introduction to utilizing SQL and PowerBI for data analysis and visualization. Throughout this guide, we've covered the basics of retrieving and processing data using SQL, followed by steps to leverage this data for creating insightful visualizations in PowerBI. To facilitate further learning and experimentation, the Jupyter notebook used in this tutorial, along with all the data files, are available for download. The .pbix file, which includes the PowerBI dashboard created in this tutorial, is also accessible. All these resources can be found in the GitHub repository, enabling you to explore, modify, and expand upon what we've discussed here.